# **Agents for Real — Plan. Act. Deliver.**

**LangChain · LangFlow · MCP (safe)**

> 25‑minute live tour + code

### Agenda
- Why agents (vs. prompts)
- Patterns: ReAct, Planner/Executor, Graph
- LangChain vs LangFlow: code vs visual
- MCP: safe, standard tools
- Live demo + micro‑eval
- Risks & guardrails

## Big idea
**Tools > Prompts.** Agents plan, call tools, and verify.

**Production focus:** versioned graphs, safe tools, evals, observability.

## Agentic patterns
- **ReAct** — think/act/observe
- **Planner/Executor** — global plan across steps
- **Graph / Multi‑agent** — parallel skills, routing, shared state

## LangChain vs LangFlow (practical)
- **LangChain** (code‑first): LCEL, tests, CI/CD
- **LangFlow** (visual): DAGs, tweak params, export JSON, REST/MCP endpoints
- **Workflow**: ideate in LangFlow → codify in LangChain

## MCP (Model Context Protocol)
- Standardizes **tool servers** (stdio / streamable‑HTTP)
- Safer by design: allow‑list, input validation, stateless by default
- Works with LangChain and LangFlow

## Risks & guardrails
- Prompt injection & untrusted tool output → sanitize & validate
- Secrets & PII → never echo; vault‑backed envs
- Observability & eval → task‑level KPIs, logging, replay

## Live demo — what you'll run
1) Build a tiny **ReAct** agent (retriever + calculator)
2) Call a **LangFlow** flow via REST
3) Attach **MCP** tools to the same agent

### Setup (one time)
```bash
pip install -U langchain langchain-openai langchain-community langchain-ollama pandas requests matplotlib
pip install -U langflow "mcp[cli]" langchain-mcp-adapters
# (Optional) Ollama: install and `ollama pull llama3.1:8b`
```
In the next cell, set `BACKEND = "OPENAI"` or `"OLLAMA"`.

In [ ]:
# Backend config — set ONE line
BACKEND = "OLLAMA"  # or "OPENAI"
MODEL_OPENAI = "gpt-4o-mini"
MODEL_OLLAMA = "llama3.1:8b"  # ensure it's pulled if using Ollama

import os, json, re, math, requests, ast, operator
from typing import List, Dict, Any

# LangChain imports
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# LLM selection
try:
    if BACKEND == "OPENAI":
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=MODEL_OPENAI, temperature=0.2)
    else:
        try:
            from langchain_community.chat_models import ChatOllama
        except Exception:
            from langchain_community.llms import Ollama as ChatOllama
        llm = ChatOllama(model=MODEL_OLLAMA)
except Exception as e:
    raise RuntimeError(f"Could not initialize LLM for BACKEND={BACKEND}: {e}")


In [ ]:
# Tiny policy corpus
POLICIES = [
    {
        "id": "TravelPlus-2024",
        "text": (
            "TravelPlus provides coverage for business travel including delayed flights, lost baggage, "
            "and emergency medical expenses up to CHF 10,000. Personal electronics are covered if "
            "they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."
        ),
    },
    {
        "id": "DeviceCare-Pro",
        "text": (
            "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. "
            "Loss or theft is covered only when a police report is filed within 48 hours. "
            "Maximum payout CHF 1,500 per device, 2 incidents per policy year."
        ),
    },
    {
        "id": "FleetAssist",
        "text": (
            "FleetAssist covers rented vehicles during company travel. "
            "It excludes personal property inside the vehicle unless explicitly endorsed."
        ),
    },
]

def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

def search_policies(query: str) -> str:
    hits = simple_search(query, top_k=3)
    return json.dumps({"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]})

# Safe arithmetic evaluator
OPS = {
    ast.Add: operator.add, ast.Sub: operator.sub,
    ast.Mult: operator.mul, ast.Div: operator.truediv,
    ast.Pow: operator.pow, ast.USub: operator.neg,
    ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv
}
def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.BinOp): return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp): return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")

def calculator(expression: str) -> str:
    try:
        node = ast.parse(expression, mode='eval').body
        return str(_eval(node))
    except Exception as e:
        return f"Error: {e}"

tools = [
    Tool(
        name="search_policies",
        func=search_policies,
        description="Search a tiny policy corpus. Input: a short query string. Returns JSON results."
    ),
    Tool(
        name="calculator",
        func=calculator,
        description="Evaluate basic arithmetic like '2*(1500-200)'. Input: expression string. Returns number as text."
    )
]


In [ ]:
# Build the ReAct-style agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

demo_questions = [
    "My work smartphone was stolen on a business trip. Is it covered, and what are the conditions?",
    "If a laptop worth CHF 1,800 is accidentally damaged twice in a year under DeviceCare-Pro, what's the max total payout after any deductibles?",
    "Does FleetAssist cover personal belongings inside a rental car?"
]
print("Demo questions ready:", *demo_questions, sep="\n- ")


In [ ]:
# Try Q1 live
res = agent.invoke(demo_questions[0])
print("\nFINAL:", res["output"] if isinstance(res, dict) and "output" in res else res)


In [ ]:
# Try Q2 live (should call calculator)
res2 = agent.invoke(demo_questions[1])
print("\nFINAL:", res2["output"] if isinstance(res2, dict) and "output" in res2 else res2)


In [ ]:
# Try Q3 live
res3 = agent.invoke(demo_questions[2])
print("\nFINAL:", res3["output"] if isinstance(res3, dict) and "output" in res3 else res3)


In [ ]:
# Micro-evaluator (2 quick checks)
import pandas as pd
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    def display_dataframe_to_user(name, df):
        print(f"\n[name={name}]\n{df}")

def contains_all(text: str, phrases: list[str]) -> bool:
    low = text.lower()
    return all(p.lower() in low for p in phrases)

tests = [
    {"q": demo_questions[0], "must": ["covered", "theft", "police", "48"]},
    {"q": demo_questions[2], "must": ["not", "unless", "endorsed"]},
]
rows = []
for t in tests:
    out = agent.invoke(t["q"])
    ans = out["output"] if isinstance(out, dict) and "output" in out else str(out)
    rows.append({"question": t["q"], "passed": contains_all(ans, t["must"]), "answer": ans})

df = pd.DataFrame(rows)
display_dataframe_to_user("LangChain agent micro‑eval", df)
df


## LangFlow — visual builder
- Start: `langflow run --host 127.0.0.1 --port 7860`
- Build: Chat Model → Prompt → (optional) Tool → Agent → **Play**
- Call from Python via REST (next cell)

In [ ]:
# Helper: call a LangFlow flow via REST
import requests, json

def run_langflow(flow_id: str, user_input: str, base_url: str = "http://127.0.0.1:7860/api/v1"):
    url = f"{base_url}/run/{flow_id}"
    payload = {"input_value": user_input, "output_type": "chat", "input_type": "chat"}
    r = requests.post(url, json=payload, timeout=60)
    r.raise_for_status()
    data = r.json()
    try:
        return data["outputs"][0]["outputs"][0]["results"]["message"]["text"]
    except Exception:
        return json.dumps(data, indent=2)

# Example:
# print(run_langflow("YOUR_FLOW_ID", "Hello from the notebook!"))


## MCP — safe tool server
We’ll write a **minimal safe server** and run it separately, then attach its tools to our agent.

In [ ]:
# Write a safe MCP server to disk (run this once)
mcp_server_code = r'''
from __future__ import annotations
from mcp.server.fastmcp import FastMCP
import re, ast, operator, json

OPS = {
    ast.Add: operator.add, ast.Sub: operator.sub,
    ast.Mult: operator.mul, ast.Div: operator.truediv,
    ast.Pow: operator.pow, ast.USub: operator.neg,
    ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv
}
def _eval(node):
    if isinstance(node, ast.Num): return node.n
    if isinstance(node, ast.BinOp): return OPS[type(node.op)](_eval(node.left), _eval(node.right))
    if isinstance(node, ast.UnaryOp): return OPS[type(node.op)](_eval(node.operand))
    raise ValueError("Unsupported expression")

def safe_calculator(expression: str) -> str:
    node = ast.parse(expression, mode='eval').body
    return str(_eval(node))

POLICIES = [
    {"id": "TravelPlus-2024", "text": "TravelPlus provides coverage for business travel including delayed flights, lost baggage, and emergency medical expenses up to CHF 10,000. Personal electronics are covered if they are lost due to theft, with a deductible of CHF 200. Pure misplacement is not covered."},
    {"id": "DeviceCare-Pro", "text": "DeviceCare-Pro covers accidental damage to smartphones and laptops used for work. Loss or theft is covered only when a police report is filed within 48 hours. Maximum payout CHF 1,500 per device, 2 incidents per policy year."},
    {"id": "FleetAssist", "text": "FleetAssist covers rented vehicles during company travel. It excludes personal property inside the vehicle unless explicitly endorsed."},
]
def simple_search(query: str, top_k: int = 3):
    terms = [t.lower() for t in re.findall(r"\w+", query)]
    scored = []
    for doc in POLICIES:
        text = doc["text"].lower()
        score = sum(text.count(t) for t in set(terms))
        if score > 0:
            scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in scored[:top_k]]

mcp = FastMCP("SR-Policies", stateless_http=True)
MAX_Q = 200
BANNED = re.compile(r"(?i)(rm\s|-rf|\bimport\b|__|eval\(|exec\()")

@mcp.tool()
def secure_search_policies(query: str) -> dict:
    if not query or len(query) > MAX_Q or BANNED.search(query or ""):
        return {"error": "query_rejected"}
    hits = simple_search(query, top_k=3)
    return {"results": [{"id": h["id"], "snippet": h["text"][:280]} for h in hits]}

@mcp.tool()
def safe_calc(expression: str) -> str:
    if len(expression or "") > 100 or BANNED.search(expression or ""):
        return "error: expression_rejected"
    try:
        return safe_calculator(expression)
    except Exception as e:
        return f"error: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http", host="127.0.0.1", port=3001)
'''
server_path = "/mnt/data/mcp_safe_server.py"
with open(server_path, "w", encoding="utf-8") as f:
    f.write(mcp_server_code)
print("Wrote MCP server to:", server_path, "\nRun in a terminal: python /mnt/data/mcp_safe_server.py  # http://127.0.0.1:3001/mcp/")


In [ ]:
# Attach MCP tools to the existing LangChain agent
# pip install -U langchain-mcp-adapters "mcp[cli]"
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import initialize_agent, AgentType
import asyncio

async def load_mcp_toolset():
    client = MultiServerMCPClient({
        "sr_policies": {"transport": "streamable_http", "url": "http://127.0.0.1:3001/mcp/"}
    })
    return await client.get_tools()

try:
    mcp_tools = asyncio.run(load_mcp_toolset())
    combined_tools = tools + mcp_tools
    agent_mcp = initialize_agent(
        tools=combined_tools, llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True, handle_parsing_errors=True,
    )
    res = agent_mcp.invoke("Calculate total payout if two CHF 1,800 laptop claims apply. Use safe_calc.")
    print("\nFINAL:", res["output"] if isinstance(res, dict) and "output" in res else res)
    print("Loaded MCP tools:", [t.name for t in mcp_tools])
except Exception as e:
    print("Note: Start the MCP server in a separate terminal before running this cell. Error:", e)


## Agent landscape (1‑year)
**Size = 50% GitHub stars (log) + 30% enterprise + 20% momentum.**
Use the next cell to regenerate or tweak.

In [ ]:
# Weighted colorful word cloud (guaranteed placement) — edit the data to tweak
import math, random, os
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

rows = [
    ("LangChain", 116_000, 0.8, 0.8),
    ("LangGraph", 19_000, 0.8, 0.9),
    ("LangFlow", 122_000, 0.6, 0.8),
    ("LlamaIndex", 44_400, 0.7, 0.7),
    ("Semantic Kernel", 26_300, 0.9, 0.8),
    ("OpenAI Agents SDK", 14_900, 0.9, 0.9),
    ("AG2 (AutoGen)", 3_600, 0.5, 0.5),
    ("CrewAI", 38_300, 0.6, 0.8),
    ("PydanticAI", 12_600, 0.6, 0.8),
    ("Smolagents (HF)", 23_000, 0.5, 0.8),
    ("Haystack", 22_700, 0.6, 0.6),
    ("Strands Agents (AWS)", 170, 0.7, 0.9),
    ("Google ADK", 200, 0.9, 0.9),
    ("Vertex AI Agent Builder", 0, 1.0, 0.9),
    ("OpenHands", 63_700, 0.5, 0.7),
    ("SuperAGI", 38_700, 0.4, 0.5),
    ("Langroid", 3_706, 0.4, 0.5),
    ("Agency Swarm", 3_400, 0.4, 0.5),
]

# scoring
logs = [math.log10(s + 10) for _, s, _, _ in rows]
mn, mx = min(logs), max(logs)
def combined(stars, ent, mom):
    star_score = (math.log10(stars + 10) - mn) / (mx - mn) if mx > mn else 0.5
    return 0.5*star_score + 0.3*ent + 0.2*mom

weights = {name: combined(stars, ent, mom) for (name, stars, ent, mom) in rows}

# palette
palette_large = [(140, 61, 43), (158, 60, 28), (169, 78, 46), (181, 82, 57)]
palette_medium = [(196, 98, 45), (210, 115, 62), (191, 107, 64), (166, 106, 63), (204, 122, 59)]
palette_small = [(74,74,74), (91,91,91), (108,108,108), (138,110,99), (127,115,107)]

def pick_color(w, wmin, wmax):
    if w >= (wmin + 0.75*(wmax - wmin)): return random.choice(palette_large)
    if w >= (wmin + 0.50*(wmax - wmin)): return random.choice(palette_medium)
    return random.choice(palette_small)

# canvas
W, H = 1800, 1000
img = Image.new("RGBA", (W, H), (255,255,255,255))
draw = ImageDraw.Draw(img)

# font
font_path = None
for p in ["/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
          "/usr/share/fonts/truetype/dejavu/DejaVuSans-Book.ttf",
          "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"]:
    if os.path.exists(p):
        font_path = p; break
if font_path is None:
    raise RuntimeError("No TTF font found.")

pairs = sorted(weights.items(), key=lambda kv: kv[1], reverse=True)
wvals = [w for _, w in pairs]; wmin, wmax = min(wvals), max(wvals)
def size_for(w): return int(18 + (w - wmin)/(wmax - wmin + 1e-9) * (120 - 18))

def inside_ellipse(x0, y0, x1, y1):
    cx, cy = W/2, H/2; rx, ry = W*0.46, H*0.38
    mx, my = (x0+x1)/2, (y0+y1)/2
    return ((mx-cx)**2)/(rx**2) + ((my-cy)**2)/(ry**2) <= 1.0

def intersects(a, b, pad=6):
    ax0, ay0, ax1, ay1 = a; bx0, by0, bx1, by1 = b
    return not (ax1+pad < bx0 or bx1+pad < ax0 or ay1+pad < by0 or by1+pad < ay0)

placed_bbs, placements = [], []
for name, w in pairs:
    base = size_for(w); color = pick_color(w, wmin, wmax); placed=False
    for shrink in range(0, 20):
        size = max(16, int(base * (0.92 ** shrink)))
        fnt = ImageFont.truetype(font_path, size=size)
        tw, th = draw.textbbox((0,0), name, font=fnt, anchor="lt")[2:4]
        a, b = 3, 6; theta = 0.0
        for _ in range(2400):
            r = a + b*theta; x = int(W/2 + r*math.cos(theta)); y = int(H/2 + r*math.sin(theta)); theta += 0.15
            x0, y0, x1, y1 = x - tw//2, y - th//2, x + tw//2, y + th//2
            if x0 < 12 or y0 < 12 or x1 > W-12 or y1 > H-12: continue
            if not inside_ellipse(x0, y0, x1, y1): continue
            if any(intersects((x0,y0,x1,y1), bb) for bb in placed_bbs): continue
            placed_bbs.append((x0,y0,x1,y1)); placements.append((name,(x,y),fnt,color)); placed=True; break
        if placed: break
    if not placed:
        fnt = ImageFont.truetype(font_path, size=16); tw, th = draw.textbbox((0,0), name, font=fnt, anchor="lt")[2:4]
        for _ in range(5000):
            import random
            x = random.randint(50, W-50); y = random.randint(70, H-50)
            x0, y0, x1, y1 = x - tw//2, y - th//2, x + tw//2, y + th//2
            if any(intersects((x0,y0,x1,y1), bb) for bb in placed_bbs): continue
            placed_bbs.append((x0,y0,x1,y1)); placements.append((name,(x,y),fnt,color)); break

for name,(x,y),fnt,color in placements:
    for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
        draw.text((x+dx, y+dy), name, font=fnt, fill=(0,0,0,35), anchor="mm")
    draw.text((x, y), name, font=fnt, fill=color, anchor="mm")

from IPython.display import display
display(img)
out = Path("/mnt/data/agents_for_real_wordcloud_generated.png")
img.convert("RGB").save(out)
print("Saved:", out)


### Presenter tips
- Keep temperature low for stability
- Remind: *the plan lives in the scratchpad; tools do the work*
- If a tool fails: describe the fallback, rerun the cell

## Closing
Agents are **tools-first** systems. Ship‑ready means:
- Versioned graphs
- Safe MCP/allow‑listed tools
- Eval + observability

**Thanks!**